In [1]:
import os
path = os.path.normpath(os.getcwd() + os.sep + os.pardir)
os.chdir(path)
path

'E:\\Semester_10\\IE506\\KFSC-Python'

In [2]:
import time
from multiprocessing import Pool
import multiprocessing
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import accuracy_score
from BestMap import BestMap
from KFSC import KFSC
from KFSC_LARGE import KFSC_LARGE
from KFSC_minibatch import KFSC_MB

C:\Users\anura\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\anura\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\anura\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
# dataset = 'mnist_sc_f150.mat'
# dataset = 'fmnist_fea_150.mat'
# dataset = 'covtype.mat'
# dataset = 'PokerHand2.mat'
# dataset = 'Postures_m.mat'
dataset = 'Epileptic.mat'
f = sio.loadmat(dataset)
X = f['X']
X = np.nan_to_num(X)
label = np.concatenate(f['Label'])
k = len(np.unique(label))

In [4]:
# Perform KFSC
tic = time.time()
opt = {
    'solver': 2,
    'maxiter': 300,
    'tol': 1e-4,
    'init_type': 'k-means-cos',
    'nrep_kmeans': 1,
    'classifier': 're'
}
lamda = 0.5
d = 30
L_kFSC, OUT = KFSC(X, k, d, lamda, opt, use_numba=False)
L_kFSC = BestMap(label[:], L_kFSC[:])
acc_kFSC = accuracy_score(label, L_kFSC)
nmi_kFSC = normalized_mutual_info_score(label, L_kFSC)
print('kFSC: acc = %.4f, nmi = %.4f' % (acc_kFSC, nmi_kFSC))
toc = time.time()
dt = toc - tic
print("Time elapsed: ", dt, " seconds")
run_info = pd.DataFrame({'Dataset': [dataset] ,'acc': [acc_kFSC],
                            'nmi': [nmi_kFSC], 'time': [dt], 'Function': 'KFSC'})
run_info.to_csv('run_info.csv', mode='a', index=False, header=False)

Solve k-FSC by accelerated Gauss-Seidel optimization ...
Initializing D by k-means algorithm (1 replicates)...
iter: 1, loss: 22709.883129, dC: 0.428874, dD: 0.035744
iter: 2, loss: 13137.726469, dC: 0.456331, dD: 0.031567
iter: 3, loss: 7359.365973, dC: 0.742089, dD: 0.042009
iter: 50, loss: 3689.996958, dC: 0.028736, dD: 0.017218
iter: 100, loss: 3680.720621, dC: 0.004623, dD: 0.003383
iter: 150, loss: 3679.094445, dC: 0.004724, dD: 0.003939
iter: 200, loss: 3676.875074, dC: 0.001457, dD: 0.001157
iter: 250, loss: 3676.807515, dC: 0.000485, dD: 0.000420
iter: 300, loss: 3676.791189, dC: 0.000209, dD: 0.000187
Max iter reached!
Assign clusters by reconstruction error
kFSC: acc = 0.4124, nmi = 0.1921
Time elapsed:  109.79012560844421  seconds


In [5]:
# Perform KFSC Large
tic = time.time()
opt = {
    'solver': 2,
    'maxiter': 300,
    'tol': 1e-4,
    'init_type': 'k-means-cos',
    'nrep_kmeans': 20,
    'classifier': 're'
}
lamda = 0.5
d = 30
L_kFSC_LARGE, OUT = KFSC_LARGE(X, k, d, lamda, opt, 500, 'k-means-cos',use_numba=False)
L_kFSC_LARGE = BestMap(label[:], L_kFSC_LARGE[:])
acc_kFSC_LARGE = accuracy_score(label, L_kFSC_LARGE)
nmi_kFSC_LARGE = normalized_mutual_info_score(label, L_kFSC_LARGE)
print('kFSC_LARGE: acc = %.4f, nmi = %.4f' % (acc_kFSC_LARGE, nmi_kFSC_LARGE))
toc = time.time()
dt = toc - tic
print("Time elapsed: ", dt, " seconds")
run_info = pd.DataFrame({'Dataset': [dataset] ,'acc': [acc_kFSC_LARGE], 
                         'nmi': [nmi_kFSC_LARGE], 'time': [dt], 'Function': 'KFSC_LARGE'})
run_info.to_csv('run_info.csv', mode='a', index=False, header=False)

Select landmark data points by k-means...
Perform kFSC on the selected landmark data points...
Solve k-FSC by accelerated Gauss-Seidel optimization ...
Initializing D by k-means algorithm (20 replicates)...
iter: 1, loss: 7471.383279, dC: 0.322410, dD: 0.033810
iter: 2, loss: 5019.666717, dC: 0.337322, dD: 0.030614
iter: 3, loss: 2718.976926, dC: 0.585894, dD: 0.033313
iter: 50, loss: 865.410379, dC: 0.025479, dD: 0.016817
iter: 100, loss: 859.965666, dC: 0.013852, dD: 0.008974
iter: 150, loss: 859.481180, dC: 0.001288, dD: 0.001035
iter: 200, loss: 859.463646, dC: 0.000534, dD: 0.000437
iter: 250, loss: 859.459127, dC: 0.000304, dD: 0.000235
iter: 300, loss: 859.457570, dC: 0.000199, dD: 0.000155
Max iter reached!
Assign clusters by reconstruction error
Predict the labels of all data points...
kFSC_LARGE: acc = 0.4041, nmi = 0.2057
Time elapsed:  128.20207953453064  seconds


In [6]:
# Perform KFSC_MB
tic = time.time()
opt = {
    'solver': 0,
    'maxiter': 300,
    'tol': 1e-4,
    'init_type': 'k-means-cos',
    'nrep_kmeans': 5,
    'classifier': 're',
    'np': 20
}
lamda = 0.5
d = 30
L_kFSC_MB, OUT = KFSC_MB(X, k, d, lamda, opt, use_numba=True)
L_kFSC_MB = BestMap(label[:], L_kFSC_MB[:])
acc_kFSC_MB = accuracy_score(label, L_kFSC_MB)
nmi_kFSC_MB = normalized_mutual_info_score(label, L_kFSC_MB)
print('kFSC_LARGE: acc = %.4f, nmi = %.4f' % (acc_kFSC_MB, nmi_kFSC_MB))
toc = time.time()
dt = toc - tic
print("Time elapsed: ", dt, " seconds")
run_info = pd.DataFrame({'Dataset': [dataset] ,'acc': [acc_kFSC_MB], 
                         'nmi': [nmi_kFSC_MB], 'time': [dt], 'Function': 'KFSC_MB'})
run_info.to_csv('run_info.csv', mode='a', index=False, header=False)

Solve k-FSC by Jacobi optimization ...
Initialise D and C by Cosine Distance k-means ...
pass 1, batch 10, sample 10001, dD 0.038202, dC 0.156592
pass 2, batch 10, sample 10001, dD 0.041937, dC 0.124844
pass 3, batch 10, sample 10001, dD 0.042756, dC 0.115030
pass 4, batch 10, sample 10001, dD 0.046569, dC 0.082147
pass 5, batch 10, sample 10001, dD 0.046481, dC 0.055554
pass 6, batch 10, sample 10001, dD 0.048406, dC 0.044839
pass 7, batch 10, sample 10001, dD 0.051885, dC 0.026585
pass 8, batch 10, sample 10001, dD 0.052475, dC 0.017541
pass 9, batch 10, sample 10001, dD 0.052496, dC 0.011574
pass 10, batch 10, sample 10001, dD 0.053543, dC 0.011143
pass 11, batch 10, sample 10001, dD 0.048820, dC 0.008170
pass 12, batch 10, sample 10001, dD 0.051659, dC 0.008269
pass 13, batch 10, sample 10001, dD 0.054700, dC 0.006917
pass 14, batch 10, sample 10001, dD 0.054985, dC 0.006273
pass 15, batch 10, sample 10001, dD 0.052686, dC 0.005613
pass 16, batch 10, sample 10001, dD 0.054284, dC 0